<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/tps_march_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. libraries and dataset

In [ ]:
!pip install category_encoders

In [ ]:
!pip install catboost

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [34]:
from sklearn.metrics import roc_auc_score

In [23]:
traindata = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/train.csv", index_col=0)
testdata = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/test.csv", index_col=0)
submission = pd.read_csv("/content/drive/MyDrive/data/kaggle_tabplayground_mar/sample_submission.csv", index_col=0)

In [24]:
traindf = traindata.copy()
testdf = testdata.copy()

#2. encoding

In [25]:
traindf.drop(columns=['cat5', 'cat13', 'cont2', 'cont7', 'cont10'], inplace=True)
testdf.drop(columns=['cat5', 'cat13', 'cont2', 'cont7', 'cont10'], inplace=True)

In [11]:
numcols = [col for col in traindf.columns if traindf[col].dtype != 'object']
catcols = [col for col in traindf.columns if col not in numcols]

In [13]:
for col in catcols:
    print(col, "\t", traindf[col].nunique())

cat0 	 2
cat1 	 15
cat2 	 19
cat3 	 13
cat4 	 20
cat6 	 16
cat7 	 51
cat8 	 61
cat9 	 19
cat10 	 299
cat11 	 2
cat12 	 2
cat14 	 2
cat15 	 4
cat16 	 4
cat17 	 4
cat18 	 4


In [30]:
catcols2 = ['cat0', 'cat11', 'cat12', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18']
catcols3 = [col for col in catcols if col not in catcols2]

In [36]:
X = traindf.drop(columns=['target'])
y = traindf['target']

In [27]:
from category_encoders.one_hot import OneHotEncoder
onehot = OneHotEncoder(cols = catcols2, return_df=True, handle_unknown=0, drop_invariant=True)
X = onehot.fit_transform(X)
testdf = onehot.transform(testdf)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [32]:
kf = StratifiedKFold(shuffle=True, random_state=42)

In [33]:
from category_encoders.target_encoder import TargetEncoder

In [46]:
ind = kf.split(X=np.zeros(300000), y=y)
scores = []

for tr_ind, val_ind in ind:
    X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
    y_tr, y_val = y.iloc[tr_ind], y.iloc[val_ind]

    enc = TargetEncoder(cols=catcols3, return_df=True, handle_unknown=y.mean())
    X_tr = enc.fit_transform(X_tr, y_tr)
    X_val = enc.transform(X_val)

    lgbm = LGBMClassifier()
    lgbm.fit(X_tr, y_tr)
    pred = lgbm.predict_proba(X_val)[:, 1]
    scores.append(roc_auc_score(y_val, pred))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: Future

In [47]:
scores

[0.8876715063004289,
 0.888103496894598,
 0.8894566805749728,
 0.8881404017678868,
 0.8899232154608159]

In [ ]:
enc = TargetEncoder(cols=catcols3, return_df=True, handle_unknown=y.mean())
X = enc.fit_transform(X, y)
testdf = enc.transform(testdf)


In [67]:
X.to_csv("X.csv")

In [68]:
testdf.to_csv("test.csv")

In [ ]:
lgbm.fit(X, y)

In [65]:
xgbc = XGBClassifier()
xgbc.fit(X, y) 

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#3. feature generation

In [ ]:
X = pd

In [77]:
from sklearn.decomposition import PCA
pca = PCA(20)
X = pd.DataFrame(pca.fit_transform(X), index=X.index)
testdf = pd.DataFrame(pca.transform(testdf), index=testdf.index)

ValueError: ignored

In [82]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
id,,,,,,,,,,,,,,,,,,,,
0,-0.729142,0.191556,-0.476783,-0.238035,-0.499007,0.132304,-0.020962,-0.257435,0.560388,-0.286952,0.228270,0.200938,0.077807,0.013697,-0.038496,0.039144,0.230417,-0.172886,-0.064149,0.145778
1,1.185771,-0.117212,-0.264304,1.172481,0.624691,-0.870156,-0.305580,0.188648,0.303413,0.021777,-0.082846,-0.085171,-0.028639,-0.071167,-0.090677,0.000375,0.506500,-0.209603,-0.261265,-0.015194
2,-0.900448,0.191127,-0.920998,0.625336,0.643068,-0.083159,-0.002996,-0.095108,0.221936,0.116798,-0.014164,-0.098144,0.129974,-0.295217,-0.011816,-0.032903,-0.167895,0.050402,0.071898,0.030822
3,-0.772144,0.108224,-0.491037,-0.274756,-0.470758,0.090688,-0.010913,-0.130771,0.655054,0.066460,0.172831,0.018972,0.179563,-0.149270,-0.168250,0.018896,0.012580,0.135084,-0.105393,-0.105561
4,0.640608,-0.513696,-0.157929,-0.146063,-0.011917,-0.117137,-1.100399,0.328041,-0.335995,0.073723,-0.153199,-0.201454,0.005475,-0.126373,0.251043,-0.068264,0.148497,-0.128521,-0.116725,0.081548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,1.374619,-0.181054,0.091510,0.187049,-0.379384,-0.766984,-0.317002,0.315844,0.345137,0.208019,-0.043320,-0.102066,-0.129609,0.089151,0.078267,-0.013826,0.168004,0.070285,0.126656,-0.139464
499995,-0.202998,-0.926012,-0.618774,0.877355,0.646420,-0.039757,-0.114807,-0.328720,0.114921,-0.291583,0.091451,0.156965,-0.054594,0.110355,-0.018089,-0.031742,-0.290652,-0.050467,0.068633,0.114201
499996,2.094385,0.232467,0.219346,-0.773013,0.780451,-0.121908,0.236233,-0.367943,0.061195,-0.272505,0.615705,-0.176439,0.127022,-0.268777,0.222971,0.025876,0.033499,0.258292,0.147791,-0.031599


In [84]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
count,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05,3.000000e+05
mean,-9.416727e-17,3.304070e-18,-1.168776e-16,-5.427455e-17,-2.413754e-17,1.597990e-17,2.795752e-17,9.255282e-18,-5.539199e-17,-6.877369e-17,-1.086358e-17,5.542222e-17,-4.645400e-17,2.007698e-16,-1.510192e-16,-3.434922e-17,-1.025086e-16,6.147971e-17,9.300014e-18,-1.736128e-16
std,1.020277e+00,6.083420e-01,5.620376e-01,5.064956e-01,4.709973e-01,4.353251e-01,3.632534e-01,3.421492e-01,3.329905e-01,2.834936e-01,2.737067e-01,2.534544e-01,2.259227e-01,2.194813e-01,1.946103e-01,1.675917e-01,1.525658e-01,1.391489e-01,1.310957e-01,1.222416e-01
min,-1.386353e+00,-1.156541e+00,-1.005756e+00,-1.410325e+00,-1.019787e+00,-1.015558e+00,-1.192748e+00,-1.301422e+00,-1.155664e+00,-1.153877e+00,-1.211195e+00,-1.215026e+00,-4.948501e-01,-6.660134e-01,-7.619616e-01,-3.598691e-01,-5.054230e-01,-6.846862e-01,-5.575414e-01,-5.037221e-01
25%,-7.588083e-01,-5.074915e-01,-4.916651e-01,-2.833802e-01,-3.450754e-01,-1.416095e-01,-1.127620e-01,-1.507196e-01,-2.373604e-01,-1.814598e-01,-9.931775e-02,-1.025493e-01,-1.324575e-01,-1.546404e-01,-1.368864e-01,-4.252315e-02,-1.099171e-01,-9.068421e-02,-8.985158e-02,-8.267030e-02
50%,-2.030325e-01,1.915464e-01,-1.222875e-01,-1.123578e-01,-1.586051e-01,-2.822045e-02,-2.717756e-02,-3.208785e-02,-2.928678e-02,-6.352225e-03,-6.814930e-03,-4.165144e-03,-2.550678e-02,-4.865789e-03,-1.768328e-03,-1.920780e-02,-9.315001e-03,-5.326172e-04,-1.027320e-02,-3.145579e-03
75%,6.913588e-01,3.232120e-01,3.807970e-01,2.670205e-01,4.057276e-01,1.048899e-01,1.431901e-01,1.229355e-01,1.969795e-01,1.736393e-01,1.059592e-01,1.150143e-01,7.383154e-02,1.446277e-01,1.334151e-01,4.364571e-03,1.012793e-01,9.027153e-02,7.989622e-02,7.652501e-02
max,2.406330e+00,1.765964e+00,1.712436e+00,1.791864e+00,2.257932e+00,1.627650e+00,1.579488e+00,1.524148e+00,1.302963e+00,1.411767e+00,1.310323e+00,1.309605e+00,1.424143e+00,1.080685e+00,8.883201e-01,1.238195e+00,7.107899e-01,7.316113e-01,6.826537e-01,6.962313e-01


# Submission

In [50]:
submission['target'] = lgbm.predict_proba(testdf)[:, 1]

In [52]:
submission.to_csv("submission.csv")

In [66]:
submission['target'] = xgbc.predict_proba(testdf)[:, 1]
submission.to_csv("submission_xgb.csv")